In [23]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
protxt= "models/colorization_deploy_v2.prototxt"
model= "models/colorization_release_v2.caffemodel"
points= "models/pts_in_hull.npy"
#image= "models/puppy.jpg"

In [24]:
net = cv2.dnn.readNetFromCaffe(protxt, model)
pts = np.load(points)

In [25]:
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

In [26]:
image = cv2.imread('puppy.jpg')

scaled = image.astype("float32") / 255.0
lab = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)

resized = cv2.resize(lab, (224, 224))
L = cv2.split(resized)[0]
L -= 50

In [27]:
net.setInput(cv2.dnn.blobFromImage(L))
ab = net.forward()[0, :, :, :].transpose((1, 2, 0))

In [28]:
ab = cv2.resize(ab, (image.shape[1], image.shape[0]))

L = cv2.split(lab)[0]
colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)

In [29]:
colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
colorized = np.clip(colorized, 0, 1)
colorized = (255 * colorized).astype("uint8")

In [30]:
image = cv2.resize(image, (700, 700))
colorized = cv2.resize(colorized, (700, 700))
cv2.imshow("Original", image)
cv2.imshow("Restored/ colorized", colorized)
cv2.waitKey(0)
cv2.destroyAllWindows()